# FOR BODY DAMAGE
IMAGE PRE PROCESSING
1. Import The ImageDataGenerator Library

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 2. Configure ImageDataGenerator Class
Image Data Augmentation

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

# 3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [3]:
training_set = train_datagen.flow_from_directory('C:/Users/MURALI/Downloads/Dataset/Car damage/body/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('C:/Users/MURALI/Downloads/Dataset/Car damage/body/validation',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


# MODEL BUILDING
1. Importing The Model Building Libraries

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

# 2. Loading The Model

In [5]:
IMAGE_SIZE = [224, 224]

train_path = 'C:/Users/MURALI/Downloads/Dataset/Car damage/body/training'
valid_path = 'C:/Users/MURALI/Downloads/Dataset/Car damage/body/validation'

In [6]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 3s 0us/step


# 3. Adding Flatten Layer

In [7]:
for layer in vgg16.layers:
    layer.trainable = False

In [8]:
folders = glob('C:/Users/MURALI/Downloads/Dataset/Car damage/body/training/*')

In [9]:
folders

['C:/Users/MURALI/Downloads/Dataset/Car damage/body/training\\00-front',
 'C:/Users/MURALI/Downloads/Dataset/Car damage/body/training\\01-rear',
 'C:/Users/MURALI/Downloads/Dataset/Car damage/body/training\\02-side']

In [10]:
x = Flatten()(vgg16.output)

In [11]:
len(folders)

3

# 4. Adding Output Layer

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

# 5. Creating A Model Object

In [13]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

# 6. Configure The Learning Process

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# 7. Train The Model

In [16]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\MURALI\AppData\Local\Temp\ipykernel_16968\563002667.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
98/98 [==============================] - 395s 4s/step - loss: 1.0367 - accuracy: 0.5618 - val_loss: 1.2905 - val_accuracy: 0.6199
Epoch 2/25
98/98 [==============================] - 433s 4s/step - loss: 0.7470 - accuracy: 0.6956 - val_loss: 0.9275 - val_accuracy: 0.6725
Epoch 3/25
98/98 [==============================] - 445s 5s/step - loss: 0.4118 - accuracy: 0.8458 - val_loss: 1.0698 - val_accuracy: 0.6199
Epoch 4/25
98/98 [==============================] - 433s 4s/step - loss: 0.3605 - accuracy: 0.8825 - val_loss: 1.1880 - val_accuracy: 0.5906
Epoch 5/25
98/98 [==============================] - 434s 4s/step - loss: 0.2914 - accuracy: 0.8774 - val_loss: 0.9338 - val_accuracy: 0.6784
Epoch 6/25
98/98 [==============================] - 430s 4s/step - loss: 0.2178 - accuracy: 0.9265 - val_loss: 1.0047 - val_accuracy: 0.6667
Epoch 7/25
98/98 [==============================] - 440s 4s/step - loss: 0.1304 - accuracy: 0.9673 - val_loss: 1.0037 - val_accuracy: 0.6784
Epoch 8/25
98

# 8. Save The Model

In [23]:
from tensorflow.keras.models import load_model
model.save('C:/Users/MURALI/Downloads/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body')

INFO:tensorflow:Assets written to: C:/Users/MURALI/Downloads/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body\assets


INFO:tensorflow:Assets written to: C:/Users/MURALI/Downloads/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body\assets


# 9. Test The Model

In [24]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [26]:
model = load_model('C:/Users/MURALI/Downloads/Intelligent Vehicle Damage Assessment & Cost Estimator For Insurance Companies/Model/body')

In [27]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [28]:
import numpy as np

In [29]:
data = "C:/Users/MURALI/Downloads/Dataset/Car damage/body/training/00-front/0003.JPEG "
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 2s 2s/step
front
